<center> <img src = https://raw.githubusercontent.com/AndreyRysistov/DatasetsForPandas/main/hh%20label.jpg alt="drawing" style="width:400px;">

# <center> Проект: Анализ вакансий из HeadHunter
   

In [143]:
import pandas as pd
import psycopg2
import requests
from bs4 import BeautifulSoup
import plotly
import plotly.express as px

In [1]:
connection = psycopg2.connect(
    dbname=DBNAME,
    user=USER,
    host=HOST,
    password=PASSWORD,
    port=PORT
)


KeyboardInterrupt



# Юнит 3. Предварительный анализ данных

1. Напишите запрос, который посчитает количество вакансий в нашей базе (вакансии находятся в таблице vacancies). 

In [55]:
# текст запроса
query_3_1 = f'''select count(v.id)
           from public.vacancies v
'''

In [56]:
# результат запроса
vac_count = pd.read_sql_query(query_3_1, connection)
vac_count = vac_count['count'][0]
vac_count

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\1348730671.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vac_count = pd.read_sql_query(query_3_1, connection)


49197

2. Напишите запрос, который посчитает количество работодателей (таблица employers). 

In [57]:
# текст запроса
query_3_2 = f'''select count(e.id)
           from public.employers e
'''

In [58]:
# результат запроса
empl_count = pd.read_sql_query(query_3_2, connection)
empl_count = empl_count['count'][0]
empl_count

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\3525268717.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  empl_count = pd.read_sql_query(query_3_2, connection)


23501

3. Посчитате с помощью запроса количество регионов (таблица areas).

In [59]:
# текст запроса
query_3_3 = f'''select count(a.id)
           from public.areas a
'''

In [60]:
# результат запроса
areas_count = pd.read_sql_query(query_3_3, connection)
areas_count = areas_count['count'][0]
areas_count

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\3577066233.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areas_count = pd.read_sql_query(query_3_3, connection)


1362

4. Посчитате с помощью запроса количество сфер деятельности в базе (таблица industries).

In [61]:
# текст запроса
query_3_4 = f'''select count(i.id)
           from public.industries i
'''

In [62]:
# результат запроса
ind_count = pd.read_sql_query(query_3_4, connection)
ind_count = ind_count['count'][0]
ind_count

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\1270799109.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  ind_count = pd.read_sql_query(query_3_4, connection)


294

***

## Выводы по предварительному анализу данных
1. Всего размещено 49197 вакансии.
2. 23501 работодателя выставили объявения.
3. Вакансии размещены в 1362 регионах.
4. Размещённые вакансии охватывают 294 сферы деятельности.



# Юнит 4. Детальный анализ вакансий

1. Напишите запрос, который позволит узнать, сколько (cnt) вакансий в каждом регионе (area).
Отсортируйте по количеству вакансий в порядке убывания.

In [63]:
# текст запроса
query_4_1 = f'''select count(v.id) vac_cnt, a.name area_name
           from public.vacancies v LEFT JOIN public.areas a ON v.area_id=a.id --присоединяем таблицы по id региона, чтобы получить их названия
           group by a.id --группируем по номеру id региона
           ORDER BY 1 DESC --сортируем в порядке убывания
'''

In [64]:
# результат запроса
area_vac_cnt = pd.read_sql_query(query_4_1, connection)
area_vac_cnt.head(5)


C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\986410471.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  area_vac_cnt = pd.read_sql_query(query_4_1, connection)


,vac_cnt,area_name
0,5333,Москва
1,2851,Санкт-Петербург
2,2112,Минск
3,2006,Новосибирск
4,1892,Алматы


2. Напишите запрос, чтобы определить у какого количества вакансий заполнено хотя бы одно из двух полей с зарплатой.

In [65]:
# текст запроса
query_4_2 = f'''select count(v.id)
           from public.vacancies v
           WHERE v.salary_from IS NOT NULL OR v.salary_to IS NOT NULL --фильтруем данные, берём только те строки, где заполненно либо поле salary_from, либо salary_to
'''

In [66]:
# результат запроса
vac_not_null = pd.read_sql_query(query_4_2, connection)
vac_not_null

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\1990295918.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  vac_not_null = pd.read_sql_query(query_4_2, connection)


,count
0,24073


3. Найдите средние значения для нижней и верхней границы зарплатной вилки. Округлите значения до целого.

In [67]:
# текст запроса
query_4_3 = f'''select AVG(v.salary_from) sf, AVG(v.salary_to) st
           from public.vacancies v
           
'''

In [68]:
# результат запроса
avg_salary = pd.read_sql_query(query_4_3, connection)
avg_sal_fr = round(avg_salary['sf'][0]) #Среднее значение нижней границы
avg_sal_to = round(avg_salary['st'][0]) #Среднее значение верхней границы
avg_sal_fr, avg_sal_to

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\858151431.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  avg_salary = pd.read_sql_query(query_4_3, connection)


(71065, 110537)

4. Напишите запрос, который выведет количество вакансий для каждого сочетания типа рабочего графика (schedule) и типа трудоустройства (employment), используемого в вакансиях. Результат отсортируйте по убыванию количества.


In [69]:
# текст запроса
query_4_4 = f'''select v.schedule, v.employment, count(v.id)
           from public.vacancies v
           group by v.schedule, v.employment --чтобы получить уникальные пары график-тип трудоустройства, группируем данные по этим признакам одновременно
           order by count(v.id) DESC
           
'''

In [70]:
# результат запроса
work_type = pd.read_sql_query(query_4_4, connection)
work_type


C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\3235091955.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  work_type = pd.read_sql_query(query_4_4, connection)


,schedule,employment,count
0,Полный день,Полная занятость,35367
1,Удаленная работа,Полная занятость,7802
2,Гибкий график,Полная занятость,1593
3,Удаленная работа,Частичная занятость,1312
4,Сменный график,Полная занятость,940
5,Полный день,Стажировка,569
6,Вахтовый метод,Полная занятость,367
7,Полный день,Частичная занятость,347
8,Гибкий график,Частичная занятость,312
9,Полный день,Проектная работа,141


5. Напишите запрос, выводящий значения поля Требуемый опыт работы (experience) в порядке возрастания количества вакансий, в которых указан данный вариант опыта. 

In [71]:
# текст запроса
query_4_5 = f'''select v.experience exp, count(v.id)
           from public.vacancies v
           group by v.experience
           order by 2 asc
           
'''

In [72]:
# результат запроса
work_exp = pd.read_sql_query(query_4_5, connection)
work_exp

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\474591956.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  work_exp = pd.read_sql_query(query_4_5, connection)


,exp,count
0,Более 6 лет,1337
1,Нет опыта,7197
2,От 3 до 6 лет,14511
3,От 1 года до 3 лет,26152


***

## Выводы по детальному анализу вакансий
1. Наибольшее количество вакансий выставлены в  городах: Москва, Санкт-Петербург, Минск, Новосибирск, Алматы.
2. Более чем в 24 000 вакансий указана либо верхняя, либо нижняя граница зарплатных ожиданий.
3. В среднем предлагаемая заработная платана колеблется от 71065 до 110537.
4. Больше всего вакансий предполагают полную занятость с разными типами рабочего графика (полный день, удалённая работа, гибкий график)
5. Меньше всего вакансий, которые требуют более 6 лет опыта работы, больше всего - опыт работы от 1 до 3 лет

# Юнит 5. Анализ работодателей

1. Напишите запрос, который позволит узнать, какие работодатели находятся на первом и пятом месте по количеству вакансий.

In [73]:
# текст запроса
query_5_1 = f'''select e.name, count(v.id)
           from public.vacancies v
                left join public.employers e on v.employer_id = e.id --объединяем таблицы по ключу - id работодателя, чтобы получить названия компаний
           group by e.name
           order by 2 DESC
           LIMIT 5 --выводим первые 5 значений, так как нас интересуют 1 и 5 место
           
'''

In [74]:
# результат запроса
top_empl = pd.read_sql_query(query_5_1, connection)
top_empl

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\2859719041.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  top_empl = pd.read_sql_query(query_5_1, connection)


,name,count
0,Яндекс,1933
1,Ростелеком,491
2,Тинькофф,444
3,СБЕР,428
4,Газпром нефть,331


2. Напишите запрос, который для каждого региона выведет количество работодателей и вакансий в нём.
Среди регионов, в которых нет вакансий, найдите тот, в котором наибольшее количество работодателей.


In [75]:
# текст запроса
query_5_2 = f'''select a.name, count(distinct e.id) cnt_emp, count(distinct v.id) cnt_vac
           from public.areas a
                left join public.employers e on a.id=e.area -- присоединяем к таблице с регионами таблицу с работодателями ключ - регион.
                left join public.vacancies v on a.id=v.area_id --присоединяем к таблице с регионами таблицу с вакансиями ключ - регион.
           where v.id is null --оставляем только те, где нет вакансий
           group by a.id -- группируем по id региона
           order by cnt_emp desc --сортируем по убыванию количества работодателей в регионе
'''

In [76]:
# результат запроса
areas = pd.read_sql_query(query_5_2, connection)
areas

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\3375254027.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  areas = pd.read_sql_query(query_5_2, connection)


,name,cnt_emp,cnt_vac
0,Россия,410,0
1,Казахстан,207,0
2,Московская область,75,0
3,Краснодарский край,19,0
4,Ростовская область,18,0
...,...,...,...
588,Скидель,0,0
589,Белоозерск,0,0
590,Орловский,0,0
591,Павловская,0,0


3. Для каждого работодателя посчитайте количество регионов, в которых он публикует свои вакансии. Отсортируйте результат по убыванию количества.


In [77]:
# текст запроса
query_5_3 = f'''select e.name emp_name, count(distinct v.area_id) cnt_area --выводим столбцы: имя работодателя, количество уникальных id регионов вакансий
                from public.vacancies v
                 join public.employers e on v.employer_id=e.id --к таблице со всеми вакансиями присоединяем таблицу с работодателями по id работодателя
                group by e.id --группируем по id работодателей
                order by cnt_area desc
'''

In [78]:
# результат запроса
emp_areas = pd.read_sql_query(query_5_3, connection)
emp_areas

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\2479837913.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  emp_areas = pd.read_sql_query(query_5_3, connection)


,emp_name,cnt_area
0,Яндекс,181
1,Ростелеком,152
2,Спецремонт,116
3,Поляков Денис Иванович,88
4,ООО ЕФИН,71
...,...,...
14901,НПП Авиатрон,1
14902,Центр дистанционных торгов,1
14903,Городские Телекоммуникационные Системы,1
14904,"Введенский, Отель",1


4. Напишите запрос для подсчёта количества работодателей, у которых не указана сфера деятельности. 

In [79]:
# текст запроса
query_5_4 = f'''select count(e.id)
                from public.employers e
                 left join public.employers_industries ei on e.id=employer_id
                where ei.employer_id is null --выбираем только те строки, в которых не оказалось сфер деятельности
'''

In [80]:
# результат запроса
emp_no_ind = pd.read_sql_query(query_5_4, connection)
emp_no_ind

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\34022229.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  emp_no_ind = pd.read_sql_query(query_5_4, connection)


,count
0,8419


5. Напишите запрос, чтобы узнать название компании, находящейся на третьем месте в алфавитном списке (по названию) компаний, у которых указано четыре сферы деятельности. 

In [81]:
# текст запроса
query_5_5 = f'''select e.name
                from public.employers e
                 left join public.employers_industries ei on e.id=employer_id
                group by e.id
                having count(ei.industry_id)=4 --оставили только те строки, где количество сфер = 4
                order by e.name
                offset 2 --отбросили первые 2 значения
                limit 1 --вывели только 1 значение, после отброшенных
'''

In [82]:
# результат запроса
third_emp = pd.read_sql_query(query_5_5, connection)
third_emp

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\1556874130.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  third_emp = pd.read_sql_query(query_5_5, connection)


,name
0,2ГИС


6. С помощью запроса выясните, у какого количества работодателей в качестве сферы деятельности указана Разработка программного обеспечения.


In [83]:
# текст запроса
query_5_6 = f'''select count(e.id)
                from public.employers e
                 join public.employers_industries ei on e.id=employer_id
                 join public.industries i on ei.industry_id=i.id
                where i.name = 'Разработка программного обеспечения' --оставили только те строки, где сфера "Разработка программного обеспечения"
'''

In [84]:
# результат запроса
emp_program = pd.read_sql_query(query_5_6, connection)
emp_program

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\587626924.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  emp_program = pd.read_sql_query(query_5_6, connection)


,count
0,3553


7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании, вместе с количеством вакансий в этих регионах. Также добавьте строку Total с общим количеством вакансий компании. Результат отсортируйте по возрастанию количества.

Список городов-милионников надо взять [отсюда](https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8). 

Если возникнут трудности с этим задание посмотрите материалы модуля  PYTHON-17. Как получать данные из веб-источников и API. 

In [85]:
# код для получения списка городов-милионников
url = 'https://ru.wikipedia.org/wiki/%D0%93%D0%BE%D1%80%D0%BE%D0%B4%D0%B0-%D0%BC%D0%B8%D0%BB%D0%BB%D0%B8%D0%BE%D0%BD%D0%B5%D1%80%D1%8B_%D0%A0%D0%BE%D1%81%D1%81%D0%B8%D0%B8' # Определяем адрес страницы
response = requests.get(url) # Выполняем GET-запрос, содержимое ответа присваивается переменной response
page = BeautifulSoup(response.text, 'html.parser') #создаём объект BeautifulSoup
cities_list = [] #Создаем пустой список
len_cities_list = len(page.find('table', class_='standard').find('tbody').find_all('a')) #Выясняем длину списка с заголовками по тегам
#В цикле проходимся по всем заголовкам с нужными тегами, чтобы получить список
for i in range(0, len_cities_list):
    city = page.find('table', class_='standard').find('tbody').find_all('a')[i].get('title')
    cities_list.append(city)
    i+=1

cities_list = cities_list[4:] #Обрезаем полученный список, так как в него попали все заголовки(title), не только названия городов
cities_list


['Москва',
 'Санкт-Петербург',
 'Новосибирск',
 'Екатеринбург',
 'Казань',
 'Нижний Новгород',
 'Красноярск',
 'Челябинск',
 'Самара',
 'Уфа',
 'Ростов-на-Дону',
 'Краснодар',
 'Омск',
 'Воронеж',
 'Пермь',
 'Волгоград']

In [86]:
#7. Для компании «Яндекс» выведите список регионов-миллионников, в которых представлены вакансии компании,
# вместе с количеством вакансий в этих регионах.
# Также добавьте строку Total с общим количеством вакансий компании. 
# Результат отсортируйте по возрастанию количества.

# текст запроса
query_5_7 = f'''Select a.name, count(v.id) cnt
                from public.vacancies v
                    left join public.employers e on v.employer_id=e.id
                    left join public.areas a on v.area_id=a.id
                where e.name = 'Яндекс' and a.name in {tuple(cities_list)}
                group by a.id
                
                UNION ALL
                
                Select 'Total', count(v.id)
                from public.vacancies v
                    left join public.employers e on v.employer_id=e.id
                    left join public.areas a on v.area_id=a.id
                where e.name = 'Яндекс' and a.name in {tuple(cities_list)}
                
                order by cnt ASC
'''

In [87]:
# результат запроса
yandex_only = pd.read_sql_query(query_5_7, connection)
yandex_only

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\635328185.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  yandex_only = pd.read_sql_query(query_5_7, connection)


,name,cnt
0,Омск,21
1,Челябинск,22
2,Красноярск,23
3,Волгоград,24
4,Пермь,25
5,Казань,25
6,Ростов-на-Дону,25
7,Самара,26
8,Уфа,26
9,Краснодар,30


***

## Выводы по анализу работодателей
1. Самое большое количество вакансий опубликовано следующими компаниями: "Яндекс", "Ростелеком", "Тинькофф", "СБЕР", "Газпром нефть". То есть, компаниями в сфере IT, коммуникаций, банковское дело, энергетическая отрасль. Большое количество вакансий в этих сферах свидетельствует о том, что именно они активнее всего развиваются.
2. Большой разброс вакансий крупных компаний по регионам свидетельствует о том, что удалённая работа приобретает все большее значение.
3. Исходя из проведённого анализа видно, что существует достаточно много регионов, где вакансии вовсе отсутствуют. Чаще всего это города с маленьким населением.

# Юнит 6. Предметный анализ

1. Сколько вакансий имеет отношение к данным?

Считаем, что вакансия имеет отношение к данным, если в её названии содержатся слова 'data' или 'данн'.

*Подсказка: Обратите внимание, что названия вакансий могут быть написаны в любом регистре.* 


In [88]:
# текст запроса
query_6_1 = '''SELECT count(v.name)
                FROM public.vacancies v
                WHERE LOWER(v.name) LIKE '%data%'
                    OR LOWER(v.name) LIKE '%данн%'
                    
'''


In [89]:
# результат запроса
data_vac = pd.read_sql_query(query_6_1, connection)
data_vac

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\1332618449.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  data_vac = pd.read_sql_query(query_6_1, connection)


,count
0,1771


2. Сколько есть подходящих вакансий для начинающего дата-сайентиста? 
Будем считать вакансиями для дата-сайентистов такие, в названии которых есть хотя бы одно из следующих сочетаний:
* 'data scientist'
* 'data science'
* 'исследователь данных'
* 'ML' (здесь не нужно брать вакансии по HTML)
* 'machine learning'
* 'машинн%обучен%'

** В следующих заданиях мы продолжим работать с вакансиями по этому условию.*

Считаем вакансиями для специалистов уровня Junior следующие:
* в названии есть слово 'junior' *или*
* требуемый опыт — Нет опыта *или*
* тип трудоустройства — Стажировка.
 

In [90]:
# текст запроса
query_6_2 = ''' WiTH ds as
                (SELECT*
                FROM public.vacancies v
                WHERE LOWER(v.name) LIKE '%data scientist%'
                    OR LOWER(v.name) LIKE '%data science%'
                    OR LOWER(v.name) LIKE '%исследователь данных%'
                    OR (LOWER(v.name) LIKE '%ml%' AND LOWER(v.name) NOT LIKE '%html%')
                    OR LOWER(v.name) LIKE '%machine learning%'
                    OR LOWER(v.name) LIKE '%машинн%обучен%') --критерии отнесения вакансии к "данным" сохраняем отдельно, чтобы было проще применить остальную фильтрацию
                SELECT count(ds.id)
                FROM ds
                WHERE LOWER(ds.name) LIKE '%junior%'
                    OR LOWER(ds.experience) LIKE '%нет опыта%'
                    OR LOWER(ds.employment) LIKE '%стажировка%'              
'''


In [91]:
# результат запроса
junior_vac = pd.read_sql_query(query_6_2, connection)
junior_vac

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\3409672001.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  junior_vac = pd.read_sql_query(query_6_2, connection)


,count
0,51


3. Сколько есть вакансий для DS, в которых в качестве ключевого навыка указан SQL или postgres?

** Критерии для отнесения вакансии к DS указаны в предыдущем задании.*

In [92]:
# текст запроса
query_6_3 = ''' WiTH ds as
                (SELECT*
                FROM public.vacancies v
                WHERE LOWER(v.name) LIKE '%data scientist%'
                    OR LOWER(v.name) LIKE '%data science%'
                    OR LOWER(v.name) LIKE '%исследователь данных%'
                    OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%') -- применяем LIKE к 'ML', т.к. в другом регистре могут попасться вакансии "TeamLead"
                    OR LOWER(v.name) LIKE '%machine learning%'
                    OR LOWER(v.name) LIKE '%машинн%обучен%')
                SELECT count(ds.id)
                FROM ds
                WHERE ds.key_skills iLIKE '%SQL%'
                    OR ds.key_skills iLIKE '%postgres%'             
'''

In [93]:
# результат запроса
sql_vac = pd.read_sql_query(query_6_3, connection)
sql_vac

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\1651414490.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sql_vac = pd.read_sql_query(query_6_3, connection)


,count
0,201


4. Проверьте, насколько популярен Python в требованиях работодателей к DS.Для этого вычислите количество вакансий, в которых в качестве ключевого навыка указан Python.

** Это можно сделать помощью запроса, аналогичного предыдущему.*

In [94]:
# текст запроса
query_6_4 = ''' WiTH ds as
                (SELECT*
                FROM public.vacancies v
                WHERE LOWER(v.name) LIKE '%data scientist%'
                    OR LOWER(v.name) LIKE '%data science%'
                    OR LOWER(v.name) LIKE '%исследователь данных%'
                    OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                    OR LOWER(v.name) LIKE '%machine learning%'
                    OR LOWER(v.name) LIKE '%машинн%обучен%')
                SELECT count(ds.id)
                FROM ds
                WHERE ds.key_skills iLIKE '%Python%'       -- фильтрацию по ключевым навыкам так же применяем после основной фильтрации "относится к данным"      
'''

In [95]:
# результат запроса
python_vac = pd.read_sql_query(query_6_4, connection)
python_vac

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\2006311537.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  python_vac = pd.read_sql_query(query_6_4, connection)


,count
0,351


5. Сколько ключевых навыков в среднем указывают в вакансиях для DS?
Ответ округлите до двух знаков после точки-разделителя.

In [96]:
# текст запроса
query_6_5 = ''' 
                SELECT round(avg(length(v.key_skills) - length(replace(v.key_skills,'\t', ''))+1),2)
                FROM public.vacancies v
                WHERE LOWER(v.name) LIKE '%data scientist%'
                    OR LOWER(v.name) LIKE '%data science%'
                    OR LOWER(v.name) LIKE '%исследователь данных%'
                    OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                    OR LOWER(v.name) LIKE '%machine learning%'
                    OR LOWER(v.name) LIKE '%машинн%обучен%'
                 
                                   
'''

In [97]:
# результат запроса
key_skills_cnt = pd.read_sql_query(query_6_5, connection)
key_skills_cnt

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\2916164914.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  key_skills_cnt = pd.read_sql_query(query_6_5, connection)


,round
0,6.41


6. Напишите запрос, позволяющий вычислить, какую зарплату для DS в **среднем** указывают для каждого типа требуемого опыта (уникальное значение из поля *experience*). 

При решении задачи примите во внимание следующее:
1. Рассматриваем только вакансии, у которых заполнено хотя бы одно из двух полей с зарплатой.
2. Если заполнены оба поля с зарплатой, то считаем зарплату по каждой вакансии как сумму двух полей, делённую на 2. Если заполнено только одно из полей, то его и считаем зарплатой по вакансии.
3. Если в расчётах участвует null, в результате он тоже даст null (посмотрите, что возвращает запрос select 1 + null). Чтобы избежать этой ситуацию, мы воспользуемся функцией [coalesce](https://postgrespro.ru/docs/postgresql/9.5/functions-conditional#functions-coalesce-nvl-ifnull), которая заменит null на значение, которое мы передадим. Например, посмотрите, что возвращает запрос `select 1 + coalesce(null, 0)`

Выясните, на какую зарплату в среднем может рассчитывать дата-сайентист с опытом работы от 3 до 6 лет. Результат округлите до целого числа. 

In [98]:
# текст запроса

query_6_6 = '''WiTH ds as
                (SELECT*
                FROM public.vacancies v
                WHERE v.name iLIKE '%data scientist%'
                    OR v.name iLIKE '%data science%'
                    OR v.name iLIKE '%исследователь данных%'
                    OR (v.name LIKE '%ML%' AND v.name NOT iLIKE '%html%')
                    OR v.name iLIKE '%machine learning%'
                    OR v.name iLIKE '%машинн%обучен%')
                  
                SELECT ds.experience, round(avg(coalesce((salary_from+salary_to)/2, salary_from, salary_to)), 0) avg_value --применяем для вычисления средней ЗП функцию coalesce (Если оба значение не нудевые - делим пополам и вычисляем среднее; иначе - берем нижнюю границу ЗП вилки, иначе - верхнюю)
                FROM ds
                WHERE ds.salary_from is NOT NULL
                    OR ds.salary_to is NOT NULL --выбираем только те строки, где хотя бы один из столбцов зарплаты не нулевое значение
                GROUP BY ds.experience
'''

In [99]:
# результат запроса
exp_price = pd.read_sql_query(query_6_6, connection)
exp_price

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\73990575.py:2: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  exp_price = pd.read_sql_query(query_6_6, connection)


,experience,avg_value
0,От 3 до 6 лет,243115.0
1,От 1 года до 3 лет,139675.0
2,Нет опыта,74643.0


***

## Выводы по предметному анализу
1. На рынке вакансий достаточно много из них связаны с работой с данными.
2. Тем не менее, только 351 из них связана с работой на python или предполагает ее.
3. В среднем вакансии, связанные с работой с данными, предполагают владение более чем 6 ключевыми навыками. Следовательное, работодатель ожидает увидеть разностороннего сотрудника, владеющего как разными языками программирования, так и другими навыками, связанными с работой с базами данных, продуктового анализа и т.д.
4. Только 51 вакансия предполагает место для начинающего аналитика данных. Из этого следует вывод, что наличие стажировок, собственных проектов при поиске первого места работы просто необходимы.

# Общий вывод по проекту

Так как в скором времени мне самой предстоит искать работу в сфере работы с данными,  решила выяснить, какие ключевые навыки чаще всего указывают работодатели, следовательно, таким образом смогу наметить для себя образовательный маршрут

In [124]:
query_7_1 = ''' 
                SELECT replace(v.key_skills,'\t', ',') as "Ключевые навыки"
                FROM public.vacancies v
                WHERE (LOWER(v.name) LIKE '%data scientist%'
                    OR LOWER(v.name) LIKE '%data science%'
                    OR LOWER(v.name) LIKE '%исследователь данных%'
                    OR (v.name LIKE '%ML%' AND LOWER(v.name) NOT LIKE '%html%')
                    OR LOWER(v.name) LIKE '%machine learning%'
                    OR LOWER(v.name) LIKE '%машинн%обучен%')
                    AND v.key_skills IS NOT NULL
                    
                 
                                   
'''

In [125]:
key_skills = pd.read_sql_query(query_7_1, connection)
key_skills

C:\Users\Tanya\AppData\Local\Temp\ipykernel_3188\4101068889.py:1: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  key_skills = pd.read_sql_query(query_7_1, connection)


,Ключевые навыки
0,"Python,SQL,Работа в команде,OpenCV,ML"
1,"Oracle Pl/SQL,Базы данных,Machine Learning,Маш..."
2,"Python,OpenCV,Tensorflow,Computer Vision,Нейро..."
3,"Python,Flask,Linux,AWS,Git"
4,"Python,SQL,NLP,Scikit-learn,Machine Learning"
...,...
426,"Python,Numpy,PyTorch,Pandas,Математический ана..."
427,"Python,Scikit-learn,Pandas,Git,Numpy"
428,"Qt,ООП,C++,MS Visual Studio,Разработка компьют..."
429,"SCALA,SQL,Математический анализ,Статистический..."


In [149]:

#Объявляем функцию для получения списка (Серии, чтобы затем было удобно с ней работать) ключевых навыков
def get_list(colomn):
    lst_to_get = []
    
    for i in colomn:
        i = i.split(',')
        for j in i:
            lst_to_get.append(j)
    
    lst_to_get = pd.Series(lst_to_get)
    
    return lst_to_get

#Применяем функцию, получаем список ключевых навыков
key_skills_lst = get_list(get_list(key_skills['Ключевые навыки']))
key_skills_cnt = key_skills_lst.value_counts().nlargest(30) #Получаем количество упоминаний навыка в вакансиях, связанных с данными, берём 30 наиболее встречающихся

#строим график , который проиллюстрирует, какие навыки наиболее необходимы
fig = px.bar(
    key_skills_cnt, #дата
    orientation='v', #ориентация графика
    height=500, #высота
    width=1000, #ширина
    title='Топ 30 ключевых навыков, необходимых для соискателей, работающих с данными' #заголовок
)
fig.update_xaxes(title_text="Ключевые навыки")
fig.update_yaxes(title_text="Количество вакансий")
fig.update_layout(showlegend=False)
#отображаем его
fig.show()


## Выводы по проектам и дополнительным исследованиям

1. Работа над проектом показала, что крупные компании предлагают свои вакансии в самых разных уголках страны. В первую очередь, это компании, связанные с IT сферой, теле-коммуникаций и энергетической промышленности.
2. В маленьких городах вакансии практически отсутсвуют, а значит, что для специалистов из провинции выходом из ситуации будет удалённая работа.
3. Относительно вакансий, связанных с работой с данными можно сделать вывод, что самые большие зарплаты предлагают соискателям с опытом работы 3-6 лет, а вот специалисты с большим опытом работы ценятся меньше. Предполагаю, что в IT сфере засиживаться на одном месте, в одной должности - дурной тон, ведь сфера активно развивается.
4. Соискатели без опыта работы будут вынуждены столкнуться с отсутсвием достаточного количества вакансий, а значит, перед утройством на свою первую работу в IT необходимо попытаться заработать этот самый опыт на стажировках, в собственных проектах или на текущем месте работы.
5. Дополнительный анализ данных показал, что основными навыками при работе с данными работодатели считают владение языком python, SQL, машинного обучения, GIT.
6. Приблизительно одинаково работодателями ценятся владение разными библиотеками, навыками работы в разных операционных системах, умение работать с большими данными и даже привычными офисными программами (редактор для презентаций)